- Lấy ra middlename.
- Tính TF-IDF của các middlename này.

In [1]:
import time
import re
import nltk
import numpy as np
import pandas as pd

In [23]:
# Loading dataset
file = pd.read_csv("name1.csv", encoding='utf-8')

In [24]:
# Số lượng dòng, cột của file csv
print(file.shape)

(228511, 2)


In [25]:
name = file['full_name']
label = file['gender']

In [26]:
# Tiền xử lý dữ liệu cho full name
male_documents = []
female_documents = []
words = 0

for sen in range(0, len(name)):
    
    if (label[sen] == 1):
        # Remove all the special characters
        document = re.sub(r'\W', ' ', str(name[sen]))
        
        # remove all single characters
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
        # Remove single characters from the start
        document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
        # Substituting multiple spaces with single space
        document = re.sub(r'\s+', ' ', document, flags=re.I)
    
        # Removing prefixed 'b'
        document = re.sub(r'^b\s+', '', document)
    
        # Converting to Lowercase
        document = document.lower()
    
        female_documents.append(document)
        
    else:
        # Remove all the special characters
        document = re.sub(r'\W', ' ', str(name[sen]))
        
        # remove all single characters
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
        # Remove single characters from the start
        document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
        # Substituting multiple spaces with single space
        document = re.sub(r'\s+', ' ', document, flags=re.I)
    
        # Removing prefixed 'b'
        document = re.sub(r'^b\s+', '', document)
    
        # Converting to Lowercase
        document = document.lower()
    
        male_documents.append(document)

In [27]:
#print(male_documents)
#print(female_documents)

In [28]:
# Thực hiện n-gram cho từng tên theo gender
featuresets = []

from sklearn.feature_extraction.text import CountVectorizer
bigram_vectorizer = CountVectorizer(analyzer="word",ngram_range=(2, 2),token_pattern=r'\b\w+\b', min_df=1,max_features=300)

# Lấy n-gram đưa vào list featureset
# Thực hiện đối với  n-gram của male
male_gram = bigram_vectorizer.fit_transform(male_documents)
male_ngram = bigram_vectorizer.get_feature_names()
for i in range(0,len(male_ngram)):
    a = (male_ngram[i],'male')
    featuresets.append(a)

# Lấy n-gram đưa vào list featureset
# Thực hiện đối với  n-gram của female
female_ngram = bigram_vectorizer.fit_transform(female_documents)
female_ngram = bigram_vectorizer.get_feature_names()
for i in range(0,len(female_ngram)):
    a = (female_ngram[i],'female')
    featuresets.append(a)

In [29]:
len(male_ngram)
#print(female_ngram)

300

In [30]:
# Tiền xử lý dữ liệu cho full name
male_names = ''
female_names = ''
words = 0

for sen in range(0,len(name)):
    
    # Dòng này đếm số lượng ký tự
    #words += len(str(name[sen]).split())
    
    if (label[sen] == 1) and (len(str(name[sen]).split()) > 2):
        # Remove all the special characters
        female_name = re.sub(r'\W', ' ', str(name[sen]))
        
        # Substituting multiple spaces with single space
        female_name = re.sub(r'\s+', ' ', female_name, flags=re.I)
    
        # Dòng này dùng để loại bỏ Họ, Tên ra khỏi document (chỉ lấy middlename)
        female_name = female_name.split(' ',1)[1]
        female_name = female_name.rsplit(' ', 1)[0]
           
        # remove all single characters
        female_name = re.sub(r'\s+[a-zA-Z]\s+', ' ', female_name)
    
        # Remove single characters from the start
        female_name = re.sub(r'\^[a-zA-Z]\s+', ' ', female_name) 
    
        # Removing prefixed 'b'
        female_name = re.sub(r'^b\s+', '', female_name)
    
        # Converting to Lowercase
        female_name = female_name.lower()
        
            
        female_names = female_names + ' ' + female_name

    if (label[sen] == 0) and (len(str(name[sen]).split()) > 2):
    
        # Remove all the special characters
        male_name = re.sub(r'\W', ' ', str(name[sen]))
        
        male_name = male_name.strip()
        # Substituting multiple spaces with single space
        male_name = re.sub(r'\s+', ' ', male_name, flags=re.I)
    
        # Dòng này dùng để loại bỏ Họ, Tên ra khỏi document
        male_name = male_name.split(' ',1)[1]
        male_name = male_name.rsplit(' ', 1)[0]
           
        # remove all single characters
        male_name = re.sub(r'\s+[a-zA-Z]\s+', ' ', male_name)
    
        # Remove single characters from the start
        male_name = re.sub(r'\^[a-zA-Z]\s+', ' ', male_name) 
    
        # Removing prefixed 'b'
        male_name = re.sub(r'^b\s+', '', male_name)
    
        # Converting to Lowercase
        male_name = male_name.lower()
        
            
        male_names = male_names + ' ' + male_name
        
        

In [31]:
male_names = male_names.split()
female_names = female_names.split()

#print(male_names)
#print(female_names)

In [32]:
#Tạo featureset là tất cả các từ trong middlename của male và female

for i in range(0,len(female_names)):
    a = (female_names[i],'female')
    featuresets.append(a)
    
for i in range(0,len(male_names)):
    a = (male_names[i],'male')
    featuresets.append(a)
    
#print(featuresets)

In [33]:
import random 
random.shuffle(featuresets) 

In [34]:
# Tạo danh sách các middle name và label
X = []
Y = []
for i in range(0,len(featuresets)):
    X.append(featuresets[i][0])
    Y.append(featuresets[i][1])    

In [35]:
print(X[10])

mẫn


In [36]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=1500, min_df=5, max_df=0.7)
X = vectorizer.fit_transform(X).toarray()


In [37]:
#lấy các chữ xuất hiện sau khi đã tính TF-IDF
#vectorizer.get_feature_names()

In [38]:
# Chia dữ liệu training và testing
X_Train, X_Test, Y_Train, Y_Test = train_test_split(X, Y, test_size = 0.4, random_state = 0)

In [18]:
print(X)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.73283576 0.68040558 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [39]:
# Xây dựng mô hình huấn luyện bằng Logistic Regression

from sklearn.linear_model import LogisticRegression
start = time.time()

classifier_LR = LogisticRegression(fit_intercept=True, n_jobs=4)
classifier_LR.fit(X_Train, Y_Train)

end = time.time()

print("Thời gian huấn Logistic Regression: ", str(end-start), " giây")

Thời gian huấn Logistic Regression:  711.3293404579163  giây


In [40]:
# Xây dựng mô hình huấn luyện bằng NaiveBayes
from sklearn.naive_bayes import BernoulliNB

start = time.time()

classifier_NB = BernoulliNB(binarize=None)
classifier_NB.fit(X_Train, Y_Train)

end = time.time()

print("Thời gian huấn luyện Naive Bayes: ", str(end-start), " giây")

Thời gian huấn luyện Naive Bayes:  3.4926631450653076  giây


In [41]:
# Predicting the test set results with LogisticRegresion
Y_Pred_LR = classifier_LR.predict(X_Test)

# Predicting the test set results with Naive Bayes
Y_Pred_NB = classifier_NB.predict(X_Test)


In [42]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print('Accuracy LogisticRegression: ', accuracy_score(Y_Test, Y_Pred_LR))
print('Accuracy Naive Bayes: ', accuracy_score(Y_Test,Y_Pred_NB))
#print('Accuracy SVM: ', accuracy_score(Y_Test, Y_Pred_SVM))

Accuracy LogisticRegression:  0.7953863216266174
Accuracy Naive Bayes:  0.7952458410351202
